## COMPAS Analysis using Aequitas
<a id='top_cell'></a>
Imagine you have a data set like the COMPAS data set, but protected variables like ethnicity and gender are not given. Can we still test whether bias exists? In this Jupyter Notebook we try to find out. 

In previous Jupyter Notebooks we edited the cleaned COMPAS data set provided by Aequitas by adding predictions for 
- sex, using the NamSor API endpoint [parsedGenderGeoBatch](https://v2.namsor.com/NamSorAPIv2/apidoc.html#/personal/parsedGenderGeoBatch), which tries to infer the sex from a first and last name and the country code, in this case 'US'
- race, using the NamSor API endpoint [usRaceEthnicity](https://v2.namsor.com/NamSorAPIv2/apidoc.html#/personal/usRaceEthnicity), which tries to infer the ethnicity from a first and last name

Even though the `usRaceEthnicity` ethnicity categories do not match COMPAS' race exactly, it should still be useful to show possible unfairness.

Using this annotated COMPAS data set, we can now test what happens if we ignore self-reported sex and race. Unlike the original Jupyter Notebook we will be leaving out analysis for the age category, as we did not do any predictions for age.

Credits: 
- We remixed the [Aequitas' cleaned COMPAS data set](https://github.com/dssg/aequitas/blob/master/examples/data/compas_for_aequitas.csv) (originally from ProPublica).
- We remixed this Notebook from [Aequitas' Sample Notebook](https://github.com/dssg/aequitas/blob/master/docs/source/examples/compas_demo.ipynb).

Content:

- [What biases exist in my model?](#existing_biases)
    - [What is the distribution of groups, predicted scores, and labels across my dataset?](#xtab)
    - [What are bias metrics across groups?](#xtab_metrics)
    - [How do I interpret biases in my model?](#interpret_bias)
    - [How do I visualize biases in my model?](#bias_viz)

- [What levels of disparity exist between population groups?](#disparities)
    - [How does the selected reference group affect disparity calculations?](#disparity_calc)
    - [How do I interpret calculated disparity ratios?](#interpret_disp)
    - [How do I visualize disparities in my model?](#disparity_viz) 

- [How do I assess model fairness??](#fairness)
    - [How do I interpret parities?](#interpret_fairness)
    - [How do I visualize bias metric parity?](#fairness_group_viz)
    - [How do I visualize parity between groups in my model?](#fairness_disp_viz) 

In [ ]:
import pandas as pd
import seaborn as sns
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.plotting import Plot

# import warnings; warnings.simplefilter('ignore')

%matplotlib inline

In [ ]:
df = pd.read_csv("data/namsor_for_aequitas.csv")
df.head()

In [ ]:
# We don't need age
df = df.drop(['age_cat'], axis=1)
df.head()

In [ ]:
# Non String columns will lead to problems later so we have to find out if there are any...
non_attr_cols = ['id', 'model_id', 'entity_id', 'score', 'label_value', 'rank_abs', 'rank_pct']
attr_cols = df.columns[~df.columns.isin(non_attr_cols)]  # index of the columns that are
df.columns[(df.dtypes != object) & (df.dtypes != str) & (df.columns.isin(attr_cols))]

In [ ]:
# and delete them.
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

In [ ]:
df.shape

## Exploration

### Risk assessment

COMPAS produces a risk score that predicts a person's likelihood of commiting a crime in the next two years. The output is a score between 1 and 10 that maps to low, medium or high. Aequitas collapsed this to a binary prediction. A score of 0 indicates a prediction of "low" risk according to COMPAS, while a 1 indicates "high" or "medium" risk.

This categorization is based on ProPublica's interpretation of Northpointe's practioner guide:

    "According to Northpointe’s practitioners guide, COMPAS “scores in the medium and high range 
    garner more interest from supervision agencies than low scores, as a low score would suggest 
    there is little risk of general recidivism,” so we considered scores any higher than “low” to 
    indicate a risk of recidivism."

The similar obtained withe the NamSor annotated COMPAS data is similar to the one that was obtained by Aequitas for the original data: We see a large difference in how risk scores are distributed by race, with a majority of white and Hispanic people predicted as low risk (score = 0) and a majority of black people predicted high and medium risk (score = 1). We don't see obviousely distinct score distributions by sex.

In [ ]:
aq_palette = sns.diverging_palette(225, 35, n=2)

In [ ]:
# >>> RACE

In [ ]:
by_race = sns.countplot(x="race", hue="score", data=df, palette=aq_palette)

In [ ]:
# >>> SEX

In [ ]:
by_sex = sns.countplot(x="sex", hue="score", data=df, palette=aq_palette)

### Levels of recidivism

This dataset includes information about whether or not the subject recidivated, and so we can directly test the accuracy of the predictions. First, we visualize the recidivsm rates across race. 

Following ProPublica, we defined recidivism as a new arrest within two years. (If a person recidivates, `label_value` = 1). They "based this decision on Northpointe’s practitioners guide, which says that its recidivism score is meant to predict 'a new misdemeanor or felony offense within two years of the COMPAS administration date.'"

Again, the results look rather similar to the original graphs, but the key message stays the same: The graphs show the recidivism rates are higher for black defendants compared to white defendants. 

In [ ]:
label_by_race = sns.countplot(x="race", hue="label_value", data=df, palette=aq_palette)

In [ ]:
label_by_sex = sns.countplot(x="sex", hue="label_value", data=df, palette=aq_palette)

[Back to Top](#top_cell)

## Analysis

Do the patterns found in the exploration reflect bias or not. 

<a id='existing_biases'></a>
### What biases exist in my model?

#### What is the distribution of groups, predicted scores, and labels across my dataset?

We will first be looking at the confusion matrix of each subgroup, calculating commonly used metrics such as false positive rate and false omission rate, as well as counts by group and group prevelance among the sample population. 

The results we get using NamSor's predictions are a bit different at times than the results we can get from the original data. For instance the tpr for Hispanics is 0.44 originally and 0.52 here. However, for now it appears that the bigger picture is still similar. African-Americans have a fpr of 42%, while Caucasians have a fpr of only 28%. This means that African-American people are far more likely to be falsely labeled as high-risk than white people. The distance between fpr for African-Americans and Caucasians appears to be smaller than for the original data. The distance between False ommision rates (`for`) seem to be even smaller here than in the original data set and the distance between false discovery rates (`fdr`) is even 0.

In [ ]:
g = Group()
xtab, _ = g.get_crosstabs(df)
absolute_metrics = g.list_absolute_metrics(xtab)

In [ ]:
# Calculated counts across sample population groups
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

In [ ]:
# Calculated rates for each sample population group
xtab[['attribute_name', 'attribute_value'] + absolute_metrics].round(2)

[Back to Top](#top_cell)
<a id='bias_viz'></a>

#### Visualization

The results we obtain when looking at group metrics accross all groups are similar to the original results:

FNR: The groups COMPAS incorrectly predicts as 'low' or 'medium' risk most often are Male and African American (see the absolute numbers). From the darker coloring, we can also tell that these are the two largest populations in the data set.

PPR: We can see that the largest 'race' group, African Americans, are still predicted positive more often than any other race group (0.56) even though this is less than in the original data set. 

FPR: African Americans are still more likely to be incorrectly classified as 'high' risk (0.42, a little less than in the original data set) than incorrectly classified as 'low' or 'medium' risk. 

We can also see that the model is equally likely to incorrectly predict a woman as 'high' risk as it is for a man (false positive rate `FPR` of 0.32 for both Male and Female).

In [ ]:
aqp = Plot()

In [ ]:
# group metrics accross all groups
a = aqp.plot_group_metric_all(xtab, ncols=2, min_group_size=0.05)

[Back to Top](#top_cell)
<a id='disparities'></a>

### What levels of disparity exist between population groups?



In [ ]:
b = Bias()
# TODO: Continue here

#### Disparities calculated in relation to a user-specified group for each attribute

In [ ]:
bdf = b.get_disparity_predefined_groups(xtab, original_df=df, 
                                        ref_groups_dict={'race':'Caucasian', 'sex':'Male'}, 
                                        alpha=0.05, check_significance=True, 
                                        mask_significance=True)
bdf.style

The `Bias()` class includes a method to quickly return a list of calculated disparities from the dataframe returned by the **`get_disparity_`** methods.

In [ ]:
# View disparity metrics added to dataframe
bdf[['attribute_name', 'attribute_value'] +
     b.list_disparities(bdf) + b.list_significance(bdf)].style

[Back to Top](#top_cell)
<a id='interpret_disp'></a>

### How do I interpret calculated disparity ratios?
The calculated disparities from the dataframe returned by the `Bias()` class **`get_disparity_`** methods are in relation to a reference group, which will always have a disparity of 1.0.

The differences in False Positive Rates, noted in the discussion of the `Group()` class above, are clarified using the disparity ratio (`fpr_disparity`). Black people are falsely identified as being high or medium risks 1.9 times the rate for white people. 

As seen above, False Discovery Rates have much less disparity (`fdr_disparity`), or fraction of false postives over predicted positive in a group. As reference groups have disparity = 1 by design in Aequitas, the lower disparity is highlighted by the `fdr_disparity` value close to 1.0 (0.906) for the race attribute group 'African-American' when disparities are calculated using predefined base group 'Caucasian'. Note that COMPAS is calibrated to  balance False Positive Rate and False Discovery Rates across groups.

[Back to Top](#top_cell)
<a id='disparity_calc'></a>

### How does the selected reference group affect disparity calculations?

Disparities calculated in the the Aequitas `Bias()` class based on the crosstab returned by the `Group()` class **`get_crosstabs()`** method can be derived using several different base gorups. In addition to using user-specified groups illustrated above, Aequitas can automate base group selection based on dataset characterisitcs:

#### Evaluating disparities calculated in relation to a different 'race' reference group
Changing even one attribute in the predefined groups will alter calculated disparities. When a differnet pre-defined group 'Hispanic' is used, we can see that Black people are 2.1 times more likely to be falsely identified as being high or medium risks as Hispanic people are (compared to 1.9 times more likely than white people), and even less likely to be falsely identified as low risk when compared to Hispanic people rather than white people.

In [ ]:
hbdf = b.get_disparity_predefined_groups(xtab, original_df=df, 
                                         ref_groups_dict={'race':'Hispanic', 'sex':'Male'},
                                         alpha=0.05,
                                         check_significance=True,
                                         mask_significance=False, 
                                         selected_significance=['fpr', 'for', 'fdr'])

In [ ]:
# View disparity metrics added to dataframe
hbdf[['attribute_name', 'attribute_value'] +  
     b.list_disparities(hbdf) + b.list_significance(hbdf)]

#### Disparities calculated in relation to sample population majority group (in terms of group prevalence) for each attribute
The majority population groups for each attribute ('race', 'sex', 'age_cat') in the COMPAS dataset are 'African American', 'Male', and '25 - 45'. Using the **`get_disparity_major_group()`** method of calculation allows researchers to quickly evaluate how much more (or less often) other groups are falsely or correctly identified as high- or medium-risk in relation to the group they have the most data on.

In [ ]:
majority_bdf = b.get_disparity_major_group(xtab, original_df=df)

In [ ]:
majority_bdf[['attribute_name', 'attribute_value'] +  b.list_disparities(majority_bdf)]

#### Disparities calculated in relation to the minimum value for each metric

When you do not have a pre-existing don’t frame of reference or policy context for the dataset (ex: Caucasians or males historically favored), you may choose to view disparities in relation to the group with the lowest value for every disparity metric, as then every group's value will be at least 1.0, and relationships can be evaluated more linearly.


Note that disparities are much more varied, and may have larger magnitude, when the minimum value per metric is used as a reference group versus one of the other two methods.

In [ ]:
min_metric_bdf = b.get_disparity_min_metric(df=xtab, original_df=df,
                                            check_significance=True)
min_metric_bdf.style

[Back to Top](#top_cell)
<a id='disparity_viz'></a>

## How do I visualize disparities in my model?
To visualize disparities in the dataframe returned by one of the `Bias()` class **`get_disparity_`** methods use one of two methods in the Aequitas `Plot()` class:

A particular disparity metric can be specified with **`plot_disparity()`**. To plot a single disparity, a metric and an attribute must be specified.

Disparities related to a list of particular metrics of interest or `'all'` metrics can be plotted with **`plot_disparity_all()`**.  At least one metric or at least one attribute must be specified when plotting multiple disparities (or the same disparity across multiple attributes). For example, to plot PPR and and Precision disparity for all attributes, specify `metrics=['ppr', 'precision']` with no attribute specified, and to plot default metrics by race, specify `attributes=['race']` and with no metrics specified.

**Reference groups are displayed in grey, and always have a disparity = 1.** Note that disparities greater than 10x reference group will are visualized as 10x, and disparities less than 0.1x reference group are visualized as 0.1x.

Statistical siginificance (at a default value of 0.05) is denoted by two asterisks (**) next to a treemap square's value.

### Visualizing disparities between groups in a single user-specified attribute for a single user-specified disparity metric

The treemap below displays precision disparity values calculated using a predefined group, in this case the 'Caucasian' group within the race attribute, sized based on the group size and colored based on disparity magnitude. We can see from asterisks that the disparities between the 'Caucasian' reference population group and both the 'African-American' and 'Other' race population groups are statistically significant at the 5% level.

**Note**: Groups are visualized at no less than 0.1 times the size of the reference group, and no more than 10 times the size of the reference group.

In [ ]:
aqp.plot_disparity(bdf, group_metric='fpr_disparity', attribute_name='race', significance_alpha=0.05)

When another group, 'Hispanic', is the reference group, the colors change to indicate higher or lower disparity in relation to that group. Treemap square sizes may also be adjusted, as group size limits for visualization are in relation to the reference group (minimum 0.1 times reference group size and maximum 10 times the reference group size).

In [ ]:
aqp.plot_disparity(hbdf, group_metric='fpr_disparity', attribute_name='race', significance_alpha=0.05)

### Visualizing disparities between all groups for a single user-specified disparity metric

The treemaps belows display False Positive Rate disparities calculated based on predefined reference groups ('race' attribute: Hispanic, 'sex' attribute: Male, 'age_cat' attribute: 25-45), sized based on group size, and colored based on disparity magnitude. 

It is clear that the majority of samples in the data are African-American, male, and 25-45 for the 'race', 'sex', and age category attributes, respectively. Based on the lighter colors in the treemaps, we see that there is precision disparity relatively close to 1 (a disparity of 1 indicates no disparity) across all attributes.

In [ ]:
j = aqp.plot_disparity_all(majority_bdf, metrics=['precision_disparity'], significance_alpha=0.05)

### Visualizing disparities between groups in a single user-specified attribute for default metrics
##### Default Metrics
When visualizing more than one disparity, you can specify a list of disparity metrics, `'all'` disaprity metrics, or use the Aequitas default disparity metrics by not supplying an argument:
- Predicted Positive Group Rate Disparity (pprev_disparity),
- Predicted Positive Rate Disparity (ppr_disparity),
- False Discovery Rate Disparity (fdr_disparity),
- False Omission Rate Disparity (for_disparity)
- False Positive Rate Disparity (fpr_disparity)
- False Negative Rate Disparity (fnr_disparity)

The treemaps below display the default disparities between 'race' groups calculated based on the minimum value of each metric, colored based on disparity magnitude. We can see based on coloring that there is a lower level of false discovery rate disparity ('fdr_disparity') between age categories than predicted positive group rate disparity or ('pprev_disparity') predicted positive rate disparity ('ppr_disparity').

In [ ]:
min_met = aqp.plot_disparity_all(min_metric_bdf, attributes=['race'], significance_alpha=0.05)

### Visualizing disparities between groups in a single user-specified attribute for all calculated disparity metrics

The treemaps below display disparities between 'race' attribute groups calculated based on predefined reference groups ('race' attribute: Hispanic, 'sex' attribute: Male) for all 10 disparity metrics, colored based on disparity magnitude.

In [ ]:
tm_capped = aqp.plot_disparity_all(hbdf, attributes=['race'], metrics = 'all', significance_alpha=0.05)

### Visualizing disparity between all groups for multiple user-specified disparity metrics

The treemaps below display False Omission Rate and False Positive Rate disparities (calculated in relation to the sample majority group for each attribute) between groups acorss all three attributes, colored based on disparity magnitude.

We see that several groups (Asian, Native American) have a much lower false omission rate than African Americans, with fairly close false omission rates between the sexes and the two older oldest age groups. Though there are many more men in the sample, the two groups have nearly identical false positive rates, while color tells us that there are larger false positive rate disparities between races and age categories than false omission rate disparity.

In [ ]:
dp = aqp.plot_disparity_all(majority_bdf, metrics=['for_disparity', 'fpr_disparity'], significance_alpha=0.05)

[Back to Top](#top_cell)
<a id='fairness'></a>

## How do I assess model fairness?

### _Aequitas Fairness() Class_
Finally, the Aequitas `Fairness()` class provides three functions that provide a high level summary of fairness. This class builds on the dataframe returned from one of the three `Bias()` class **`get_dispariy_`** methods. 

Using FPR disparity as an example and the default fairness threshold, we have:

$$ 0.8 < Disparity_{FNR} =  \frac{FPR_{group}}{FPR_{base group}} < 1.25 $$ 

We can assess fairness at various levels of detail:

### Group Level Fairness
When the `label_value` column is not included in the original data set, Aequitas calculates only Statistical Parity and Impact Parities.

When the `label_value` is included in the original data set, the **`get_group_value_fairness()`** function builds on the previous dataframe. The **`get_group_value_fairness()`**
function gives us attribute group-level statistics for fairness determinations:

#### Pairities Calcuated:

| Parity | Column Name |
| --- | --- |
| True Positive Rate Parity | 'TPR Parity' |
| True Negative Rate Parity | 'TNR Parity' |
| False Omission Rate Parity | 'FOR Parity' |
| False Discovery Rate Parity | 'FDR Parity' |
| False Positive Rate Parity | 'FPR Parity' |
| False Negative Rate Parity | 'FNR Parity' |
| Negative Predictive Value Parity | 'NPV Parity' |
| Precision Parity | 'Precision Parity' |
| Predicted Positive Ratio$_k$ Parity | 'Statistical Parity' |
| Predicted Positive Ratio$_g$ Parity | 'Impact Parity' |

#### Also assessed:
- **_Type I Parity_**: Fairness in both FDR Parity and FPR Parity
- **_Type II Parity_**: Fairness in both FOR Parity and FNR Parity
- **_Equalized Odds_**: Fairness in both FPR Parity and TPR Parity
- **_Unsupervised Fairness_**: Fairness in both Statistical Parity and Impact Parity
- **_Supervised Fairness_**: Fairness in both Type I and Type II Parity
- **_Overall Fairness_**: Fairness across all parities for all attributes

In [ ]:
f = Fairness()
fdf = f.get_group_value_fairness(bdf)

The `Fairness()` class includes a method to quickly return a list of fairness determinations from the dataframe returned by the **`get_group_value_fairness()`** method.

In [ ]:
parity_detrminations = f.list_parities(fdf)

In [ ]:
fdf[['attribute_name', 'attribute_value'] + absolute_metrics + b.list_disparities(fdf) + parity_detrminations].style

[Back to Top](#top_cell)
<a id='interpret_fairness'></a>

### How do I interpret parities?
Calling the Aequitas `Fairness()` class **`get_group_value_fairness()`** method on the dataframe returned from a `Bias()` class `get_dispariy` method will return the dataframe with additional columns indicating parities, as seen in the slice of the `get_group_value_fairness` data frame directly above.

In this case, our base groups are Caucasian for race, Male for gender, and 25-45 for age_cat. By construction, the base group has supervised fairness. (The disparity ratio is 1). Relative to the base groups, the COMPAS predictions only provide supervised fairness to one group, Hispanic.

Above, the African-American false omission and false discovery are within the bounds of fairness. This result is expected because COMPAS is calibrated. (Given calibration, it is surprising that Asian and Native American rates are so low. This may be a matter of having few observations for these groups.)

On the other hand, African-Americans are roughly twice as likely to have false positives and 40 percent less likely to false negatives. In real terms, 44.8% of African-Americans who did not recidivate were marked high or medium risk (with potential for associated penalties), compared with 23.4% of Caucasian non-reoffenders. This is unfair and is marked False below.

These findings mark an inherent trade-off between FPR Fairness, FNR Fairness and calibration, which is present in any decision system where base rates are not equal. See [Chouldechova (2017)](https://www.andrew.cmu.edu/user/achoulde/files/disparate_impact.pdf). Aequitas helps bring this trade-off to the forefront with clear metrics and asks system designers to make a reasoned decision based on their use case.

### Attribute Level Fairness
Use the **`get_group_attribute_fairness()`** function to view only the calculated parities from the **`get_group_value_fairness()`** function at the attribute level.

In [ ]:
gaf = f.get_group_attribute_fairness(fdf)
gaf

### Overall Fairness
The **`get_overall_fairness()`** function gives a quick boolean assessment of the output of **`get_group_value_fairness()`** or **`get_group_attribute_fairness()`**, returning a dictionary with a determination across all attributes for each of:
- Unsupervised Fairness
- Supervised Fairness
- Overall Fairness

In [ ]:
gof = f.get_overall_fairness(fdf)
gof

[Back to Top](#top_cell)
<a id='fairness_group_viz'></a>

## How do I visualize bias metric parity?
Once you have run the `Group()` class to retrieve a crosstab of absolute group value bias metrics, added calculdated disparities via one of the the `Bias()` class **`get_disparity`** functions, and added parity determinations via the `Fairness()` class **`get_group_value_fairness()`** or **`get_group_attribute_fairness()`** method, you are ready to visualize biases and disparities in terms of fairness determination.

For visualizing absolute metric fairness with the the Aequitas `Plot()` class, a particular metric can be specified with **`plot_fairness_group()`**. A list of particular metrics of interest or 'all' metrics can be plotted with **`plot_fairness_group_all()`**.

### Visualizing parity of a single absolute group metric across all population groups

The chart below displays absolute group metric Predicted Positive Rate Disparity (ppr) across each attribute, colored based on fairness determination for that attribute group (green = 'True' and red = 'False'). 

We can see from the green color that only the 25-45 age group, Male category, and Caucasian groups have been determined to be fair. Sound familiar? They should! These are the groups selected as reference groups, so this model is not fair in terms of Statistical Parity for any of the other groups.

In [ ]:
z = aqp.plot_fairness_group(fdf, group_metric='ppr')

### Visualizing all absolute group metrics across all population groups
The charts below display all calculated absolute group metrics across each attribute, colored based on fairness determination for that attribute group (green = 'True' and red = 'False'). 

Immediately we can see that negative predictive parity status is 'True' for all population groups, and that only two groups had a 'False' determination for true negative parity. 

In [ ]:
fg = aqp.plot_fairness_group_all(fdf, ncols=5, metrics = "all")

[Back to Top](#top_cell)
<a id='fairness_disp_viz'></a>

## How do I visualize parity between groups in my model? 
To visualize disparity fairness based on the dataframe returned from the Fairness() class **`get_group_value_fairness()`** method, a particular disparity metric can be specified with the **`plot_fairness_disparity()`** method in the the Aequitas `Plot()` class. To plot a single disparity, a metric and an attribute must be specified.

Disparities related to a list of particular metrics of interest or `'all'` metrics can be plotted with **`plot_fairness_disparity_all()`**. At least one metric or at least one attribute **must** be specified when plotting multiple fairness disparities (or the same disparity across multiple attributes).

### Visualizing parity between groups in a single user-specified attribute for all calculated disparity metrics

The treemap below displays False Discovery Rate disparity values between race attribute groups calculated based on a predefined reference group ('Caucasian'), colored based on fairness determination for that attribute group (green = 'True' and red = 'False'). We see very quickly that only two groups have a 'False' parity determination.

In [ ]:
m = aqp.plot_fairness_disparity(fdf, group_metric='fdr', attribute_name='race')

In [ ]:
fpr = aqp.plot_fairness_disparity(fdf, group_metric='fpr', attribute_name='race')

### Researcher Check: Could the unfairness I am seeing be related to small group sizes in my sample?

Use the `min_group` parameter on all visualization methods to vizualize parities for only those sample population groups above a user-specified percentage of the total sample size. Note that only the smallest groups had an 'False' determination for false discovery rate parity above. The parity determination is 'True' for all groups at least 1% of the sample size .

In [ ]:
m = aqp.plot_fairness_disparity(fdf, group_metric='fdr', attribute_name='race', 
                                min_group_size=0.01, significance_alpha=0.05)

### Visualizing parity between groups in a single user-specified attribute for all calculated disparity metrics

The treemaps below display disparities between race attribute groups calculated based on a predefined reference group ('Caucasian') for all 10 disparity metrics, colored based on fairness determination for that attribute group (green = 'True' and red = 'False').

As all treemap squares are sized and positioned based on group size, the population groups on all subplots are found in the same place across all disparity metrics, allowing for ease of comparison of fairness determinations for each 'race' group across every calculated dipsarity metric.

In [ ]:
a_tm = aqp.plot_fairness_disparity_all(fdf, attributes=['race'], metrics='all', 
                                       significance_alpha=0.05)

### Visualizing parity between all groups for multiple user-specified disparity metrics

The treemaps below display Predicted Positive Group Rate (pprev) and Predicted Positive Rate (ppr) disparities between attribute groups for all three attributes (race, sex, age category) calculated based on predefined reference groups ('race' attribute: Caucasian, 'sex' attribute: Male, 'age_cat' attribute: 25-45), colored based on fairness determination for that attribute group (green = 'True' and red = 'False'). As we want to plot for all groups, there is no need to specify any attributes. 

We can see that the Predicted Positive Group Rate Parity (Impact Parity) determination was 'False' for nearly every race in comparison to Caucausians, and 'False' for every other age category in comparison to the 25-45 age group, and that overall Predicted Positive Rate Parity (Statistical Parity) did not have any 'True' fairness determinations at all.

In [ ]:
r_tm = aqp.plot_fairness_disparity_all(fdf, metrics=['pprev_disparity', 'ppr_disparity'], 
                                       significance_alpha=0.05)

### Visualizing parity between groups in multiple user-specified attributes

The treemaps below display disparities between attribute groups for all two attributes (sex, age category) calculated based on predefined reference groups ('sex' attribute: Male, 'age_cat' attribute: 25-45) for the six default disparity metrics, colored based on fairness determination for that attribute group (green = 'True' and red = 'False'). As we want to see only the default metrics, we do not need to set the 'metrics' parameter. 

Note that there is slightly more parity between the sexes (FNR, FDR, FNR, and Statistical Parity) than between age categories (FDR Parity only).

In [ ]:
n_tm = aqp.plot_fairness_disparity_all(fdf, attributes=['sex', 'race'], 
                                       significance_alpha=0.05)

## The Aequitas Effect

By breaking down the COMPAS predictions using a variety of bias and disparity metrics calculated using different reference groups, we are able to surface the specific metrics for which the model is imposing bias on given attribute groups, and have a clearer lens when evaluating models and making recommendations for intervention. 

Researchers utilizing Aequitas will be able to make similar evaluations on their own data sets, and as they continue to use the tool, will begin to identify patterns in where biases exist and which models appear to produce less bias, thereby helping to reduce bias and its effects in future algorithm-based decision-making.

[Back to Top](#top_cell)